### Geobr

Abaixo, algumas outras bibliotecas de mapas que são de fácil interação com a Geobr. Podem ser úteis futuramente.

In [1]:
import pandas as pd
import pandas_gbq
import geopandas as gpd
import rtree
import pygeos
import mapclassify
import geobr
from shapely.geometry import Point
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import folium

C:\Users\Vinícius\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\geopandas\_compat.py:153: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  set_use_pygeos()


In [4]:
gdf = geobr.read_municipality(code_muni='all', year=2020)
rj_cidade = gdf[gdf.name_muni == 'Rio De Janeiro']
rj_cidade.explore()

In [9]:
rj_cidade.explore(
    color = 'gray',
    tooltip = 'name_muni',
    name = 'Rio de Janeiro',
    tiles = 'CartoDB positron'
)

In [3]:
query_websirene = "SELECT * FROM rj-cor.clima_pluviometro.estacoes_websirene"
query_alertario = "SELECT * FROM rj-cor.clima_pluviometro.estacoes_alertario"
query_inea = "SELECT * FROM rj-cor.clima_pluviometro.estacoes_inea"
query_cemaden = "SELECT * FROM rj-cor.clima_pluviometro.estacoes_cemaden"

df_websirene = pandas_gbq.read_gbq(query_websirene, project_id = "hackaton-fgv").dropna(subset=['latitude'], how='all')
df_alertario = pandas_gbq.read_gbq(query_alertario, project_id = "hackaton-fgv").dropna(subset=['latitude'], how='all')
df_inea = pandas_gbq.read_gbq(query_inea, project_id = "hackaton-fgv").dropna(subset=['latitude'], how='all')
df_cemaden = pandas_gbq.read_gbq(query_cemaden, project_id = "hackaton-fgv").dropna(subset=['latitude'], how='all')

geometry_websirene = [Point(lon, lat) for lon, lat in zip(df_websirene['longitude'], df_websirene['latitude'])]
gdf_websirene = gpd.GeoDataFrame(df_websirene, geometry = geometry_websirene, crs = 'EPSG:4326')

geometry_alertario = [Point(lon, lat) for lon, lat in zip(df_alertario['longitude'], df_alertario['latitude'])]
gdf_alertario = gpd.GeoDataFrame(df_alertario, geometry = geometry_alertario, crs = 'EPSG:4326')

geometry_inea = [Point(lon, lat) for lon, lat in zip(df_inea['longitude'], df_inea['latitude'])]
gdf_inea = gpd.GeoDataFrame(df_inea, geometry = geometry_inea, crs = 'EPSG:4326')

geometry_cemaden = [Point(lon, lat) for lon, lat in zip(df_cemaden['longitude'], df_cemaden['latitude'])]
gdf_cemaden = gpd.GeoDataFrame(df_cemaden, geometry = geometry_cemaden, crs = 'EPSG:4326')

# Criando um mapa interativo com Folium
m = folium.Map(location=[-22.9068, -43.1729], zoom_start=10)

# Adicionando pontos ao mapa
for idx, row in gdf_websirene.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['estacao'].title(), icon=folium.Icon(color='blue')).add_to(m)

for idx, row in gdf_alertario.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['estacao'].title(), icon=folium.Icon(color='red')).add_to(m)

for idx, row in gdf_inea.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['estacao'].title(), icon=folium.Icon(color='orange')).add_to(m)

for idx, row in gdf_cemaden.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['estacao'].title(), icon=folium.Icon(color='green')).add_to(m)

legend_html = '''
     <div style="position: fixed; bottom: 50px; right: 50px; z-index:1000; font-size:14px;">
     <p><span style="background-color: blue; padding: 5px; border-radius: 5px;"></span> Websirene</p>
     <p><span style="background-color: red; padding: 5px; border-radius: 5px;"></span> Alertário</p>
     <p><span style="background-color: orange; padding: 5px; border-radius: 5px;"></span> INEA</p>
     <p><span style="background-color: green; padding: 5px; border-radius: 5px;"></span> CEMADEN</p>
      </div>
     '''

m.get_root().html.add_child(folium.Element(legend_html))

# Salvando o mapa como HTML
m.save('mapa_interativo.html')

Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
